**<h1><center> VECTOR SPACE MODEL + BM25 </center></h1>**

#Thông tin thành viên


1.   Lê Tấn Lộc - MSSV: 20521546
2.   Lê Trần Hữu Phước - MSSV: 20521775
3.   Nguyễn Thị Ngọc Nga - MSSV: 20521641



# Load dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
#đặt đường dẫn dữ liệu
path_corpus = '/content/drive/MyDrive/CS419.N11.KHCL/Cranfield'
path_query = '/content/drive/MyDrive/CS419.N11.KHCL/TEST/query.txt'
path_relevance = '/content/drive/MyDrive/CS419.N11.KHCL/TEST/RES'

Đọc dữ liệu Cranfield vào corpus

In [ ]:
corpus = []
for path in range(1400):
  with open(path_corpus +'/'+ str(path+1) + '.txt', 'r', encoding ='UTF-8') as f:
    temp = f.readline()
    corpus.append(temp)
print(corpus[0])

experimental investigation of the aerodynamics of a wing in a slipstream . an experimental study of a wing in a propeller slipstream was made in order to determine the spanwise distribution of the lift increase due to slipstream at different angles of attack of the wing and at different free stream to slipstream velocity ratios .  the results were intended in part as an evaluation basis for different theoretical treatments of this problem . the comparative span loading curves, together with supporting evidence, showed that a substantial part of the lift increment produced by the slipstream was due to a /destalling/ or boundary layer control effect .  the integrated remaining lift increment, after subtracting this destalling lift, was found to agree well with a potential flow theory . an empirical evaluation of the destalling effects was made for the specific configuration of the experiment . 


Load query

In [ ]:
queries = []
with open(path_query, 'r', encoding='UTF-8') as f:
  values = f.readlines()
  for item in values:
    queries.append(item.split('\t')[1].strip('\n'))
print(queries[0])

what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .


Load relevant documents

In [ ]:
relevance = {}
for i in range(225):
  with open(path_relevance + '/' + str(i+1) + '.txt', 'r', encoding='UTF-8') as f:
    temp = f.readlines()
    num_query = temp[0].split()[0]
    relevant = []
    for i in range(len(temp)):
      value = temp[i].split()[1]
      value = value.split('\t')[0]
      relevant.append(int(value))

    relevance[num_query] = sorted(relevant)
print(relevance['1'])

[12, 13, 14, 15, 29, 30, 31, 37, 51, 52, 56, 57, 66, 95, 102, 142, 184, 185, 195, 378, 462, 486, 497, 858, 859, 875, 876, 879, 880]


# Stemming Function

In [ ]:
import sys

class My_Stemmer:

    def __init__(self):
        self.b = ""  # buffer for word to be stemmed
        self.k = 0
        self.k0 = 0
        self.j = 0   # j is a general offset into the string

    def cons(self, i):
        if self.b[i] == 'a' or self.b[i] == 'e' or self.b[i] == 'i' or self.b[i] == 'o' or self.b[i] == 'u':
            return 0
        if self.b[i] == 'y':
            if i == self.k0:
                return 1
            else:
                return (not self.cons(i - 1))
        return 1

    def m(self):
        n = 0
        i = self.k0
        while 1:
            if i > self.j:
                return n
            if not self.cons(i):
                break
            i = i + 1
        i = i + 1
        while 1:
            while 1:
                if i > self.j:
                    return n
                if self.cons(i):
                    break
                i = i + 1
            i = i + 1
            n = n + 1
            while 1:
                if i > self.j:
                    return n
                if not self.cons(i):
                    break
                i = i + 1
            i = i + 1

    def vowelinstem(self):
        """vowelinstem() is TRUE <=> k0,...j contains a vowel"""
        for i in range(self.k0, self.j + 1):
            if not self.cons(i):
                return 1
        return 0

    def doublec(self, j):
        """doublec(j) is TRUE <=> j,(j-1) contain a double consonant."""
        if j < (self.k0 + 1):
            return 0
        if (self.b[j] != self.b[j-1]):
            return 0
        return self.cons(j)

    def cvc(self, i):
        if i < (self.k0 + 2) or not self.cons(i) or self.cons(i-1) or not self.cons(i-2):
            return 0
        ch = self.b[i]
        if ch == 'w' or ch == 'x' or ch == 'y':
            return 0
        return 1

    def ends(self, s):
        """ends(s) is TRUE <=> k0,...k ends with the string s."""
        length = len(s)
        if s[length - 1] != self.b[self.k]: # tiny speed-up
            return 0
        if length > (self.k - self.k0 + 1):
            return 0
        if self.b[self.k-length+1:self.k+1] != s:
            return 0
        self.j = self.k - length
        return 1

    def setto(self, s):
        """setto(s) sets (j+1),...k to the characters in the string s, readjusting k."""
        length = len(s)
        self.b = self.b[:self.j+1] + s + self.b[self.j+length+1:]
        self.k = self.j + length

    def r(self, s):
        """r(s) is used further down."""
        if self.m() > 0:
            self.setto(s)

    def step1ab(self):
        if self.b[self.k] == 's':
            if self.ends("sses"):
                self.k = self.k - 2
            elif self.ends("ies"):
                self.setto("i")
            elif self.b[self.k - 1] != 's':
                self.k = self.k - 1
        if self.ends("eed"):
            if self.m() > 0:
                self.k = self.k - 1
        elif (self.ends("ed") or self.ends("ing")) and self.vowelinstem():
            self.k = self.j
            if self.ends("at"):   self.setto("ate")
            elif self.ends("bl"): self.setto("ble")
            elif self.ends("iz"): self.setto("ize")
            elif self.doublec(self.k):
                self.k = self.k - 1
                ch = self.b[self.k]
                if ch == 'l' or ch == 's' or ch == 'z':
                    self.k = self.k + 1
            elif (self.m() == 1 and self.cvc(self.k)):
                self.setto("e")

    def step1c(self):
        """step1c() turns terminal y to i when there is another vowel in the stem."""
        if (self.ends("y") and self.vowelinstem()):
            self.b = self.b[:self.k] + 'i' + self.b[self.k+1:]

    def step2(self):
        if self.b[self.k - 1] == 'a':
            if self.ends("ational"):   self.r("ate")
            elif self.ends("tional"):  self.r("tion")
        elif self.b[self.k - 1] == 'c':
            if self.ends("enci"):      self.r("ence")
            elif self.ends("anci"):    self.r("ance")
        elif self.b[self.k - 1] == 'e':
            if self.ends("izer"):      self.r("ize")
        elif self.b[self.k - 1] == 'l':
            if self.ends("bli"):       self.r("ble") # --DEPARTURE--
            # To match the published algorithm, replace this phrase with
            #   if self.ends("abli"):      self.r("able")
            elif self.ends("alli"):    self.r("al")
            elif self.ends("entli"):   self.r("ent")
            elif self.ends("eli"):     self.r("e")
            elif self.ends("ousli"):   self.r("ous")
        elif self.b[self.k - 1] == 'o':
            if self.ends("ization"):   self.r("ize")
            elif self.ends("ation"):   self.r("ate")
            elif self.ends("ator"):    self.r("ate")
        elif self.b[self.k - 1] == 's':
            if self.ends("alism"):     self.r("al")
            elif self.ends("iveness"): self.r("ive")
            elif self.ends("fulness"): self.r("ful")
            elif self.ends("ousness"): self.r("ous")
        elif self.b[self.k - 1] == 't':
            if self.ends("aliti"):     self.r("al")
            elif self.ends("iviti"):   self.r("ive")
            elif self.ends("biliti"):  self.r("ble")
        elif self.b[self.k - 1] == 'g': # --DEPARTURE--
            if self.ends("logi"):      self.r("log")
        # To match the published algorithm, delete this phrase

    def step3(self):
        """step3() dels with -ic-, -full, -ness etc. similar strategy to step2."""
        if self.b[self.k] == 'e':
            if self.ends("icate"):     self.r("ic")
            elif self.ends("ative"):   self.r("")
            elif self.ends("alize"):   self.r("al")
        elif self.b[self.k] == 'i':
            if self.ends("iciti"):     self.r("ic")
        elif self.b[self.k] == 'l':
            if self.ends("ical"):      self.r("ic")
            elif self.ends("ful"):     self.r("")
        elif self.b[self.k] == 's':
            if self.ends("ness"):      self.r("")

    def step4(self):
        """step4() takes off -ant, -ence etc., in context vcvc."""
        if self.b[self.k - 1] == 'a':
            if self.ends("al"): pass
            else: return
        elif self.b[self.k - 1] == 'c':
            if self.ends("ance"): pass
            elif self.ends("ence"): pass
            else: return
        elif self.b[self.k - 1] == 'e':
            if self.ends("er"): pass
            else: return
        elif self.b[self.k - 1] == 'i':
            if self.ends("ic"): pass
            else: return
        elif self.b[self.k - 1] == 'l':
            if self.ends("able"): pass
            elif self.ends("ible"): pass
            else: return
        elif self.b[self.k - 1] == 'n':
            if self.ends("ant"): pass
            elif self.ends("ement"): pass
            elif self.ends("ment"): pass
            elif self.ends("ent"): pass
            else: return
        elif self.b[self.k - 1] == 'o':
            if self.ends("ion") and (self.b[self.j] == 's' or self.b[self.j] == 't'): pass
            elif self.ends("ou"): pass
            # takes care of -ous
            else: return
        elif self.b[self.k - 1] == 's':
            if self.ends("ism"): pass
            else: return
        elif self.b[self.k - 1] == 't':
            if self.ends("ate"): pass
            elif self.ends("iti"): pass
            else: return
        elif self.b[self.k - 1] == 'u':
            if self.ends("ous"): pass
            else: return
        elif self.b[self.k - 1] == 'v':
            if self.ends("ive"): pass
            else: return
        elif self.b[self.k - 1] == 'z':
            if self.ends("ize"): pass
            else: return
        else:
            return
        if self.m() > 1:
            self.k = self.j

    def step5(self):
        """step5() removes a final -e if m() > 1, and changes -ll to -l if
        m() > 1.
        """
        self.j = self.k
        if self.b[self.k] == 'e':
            a = self.m()
            if a > 1 or (a == 1 and not self.cvc(self.k-1)):
                self.k = self.k - 1
        if self.b[self.k] == 'l' and self.doublec(self.k) and self.m() > 1:
            self.k = self.k -1

    def stem(self, p, i, j):
        # copy the parameters into statics
        self.b = p
        self.k = j
        self.k0 = i
        if self.k <= self.k0 + 1:
            return self.b # --DEPARTURE--

        # With this line, strings of length 1 or 2 don't go through the
        # stemming process, although no mention is made of this in the
        # published algorithm. Remove the line to match the published
        # algorithm.

        self.step1ab()
        self.step1c()
        self.step2()
        self.step3()
        self.step4()
        self.step5()
        return self.b[self.k0:self.k+1]

# Evaluation Function

In [3]:
import os
data_path = '/content/drive/MyDrive/CS419.N11.KHCL/TEST'
evaluate_path = os.path.join(data_path, 'RES')

In [4]:
class Evaluate:
    def __init__(self, evapath, predict, rank=False):
        self.dataLength = len(list(os.listdir(evapath)))
        self.predict = predict

        self.gold_standard = []
        self.load_relevant_doc()

        self.precision = []
        self.calculate_precision(rank)

        self.recall = []
        self.calculate_recall(rank)

    def load_relevant_doc(self):
        for i in range(self.dataLength):
            file_path = os.path.join(evaluate_path, str(i + 1) + '.txt')

            with open(file_path, 'r', encoding="utf-8") as file:
                data = []
                for line in file:
                    data.append(int(line.split()[1]))

                self.gold_standard.append(data)

    def calculate_precision(self, rank):
        if rank:
            for i in range(self.dataLength):
                p = []
                right = 0

                for index, res in enumerate(self.predict[i]):
                    if res in self.gold_standard[i]:
                        right += 1
                        p.append(right / (index + 1))

                self.precision.append(p)
        else:
            for i in range(self.dataLength):
                right = 0

                for res in self.predict[i]:
                    if res in self.gold_standard[i]:
                        right += 1

                self.precision.append(right / len(self.predict[i]))

    def calculate_recall(self, rank):
        if rank:
            for i in range(self.dataLength):
                r = []

                for j in range(len(self.precision[i])):
                    r.append((j + 1) / len(self.gold_standard[i]))

                self.recall.append(r)
        else:
            for i in range(self.dataLength):
                right = 0

                for res in self.predict[i]:
                    if res in self.gold_standard[i]:
                        right += 1

                self.recall.append(right / len(self.gold_standard[i]))


    def average(self, array):
        sum = 0
        count = 0
        for element in array:
            sum += element
            count += 1

        return sum / count if count else 0

    def find_P_from_R_inter(self, r, P, R):
        p = []
        for i in range(len(P)):
            if R[i] >= r:
                p.append(P[i])

        return max(p) if len(p) else 0

    def AP_no_inter(self):
        AP = []
        for res in (self.precision):
            AP.append(self.average(res))

        return AP

    def AP_inter(self):
        AP = []

        for i in range(self.dataLength):
            p = []

            for r in range(11):
                p.append(self.find_P_from_R_inter(r / 10, self.precision[i], self.recall[i]))

            AP.append(self.average(p))

        return AP

    def MAP(self, interpolate=False):
        if interpolate:
            AP = self.AP_inter()
        else:
            AP = self.AP_no_inter()

        return self.average(AP)

# Preprocessing

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
!pip install num2words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words


import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math


In [ ]:
# Chuyển toàn bộ dữ liệu thành chữ thường
def convert_lower_case(data):
    return np.char.lower(data)

In [ ]:
# Loại bỏ stopwords
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [ ]:
# Loại bỏ dấu câu
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [ ]:
# Loại bỏ dấu nháy đơn
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [ ]:
# stemming
def stemming(data):
    # stemmer= PorterStemmer()
    
    stemmer = My_Stemmer()    # my_stemming
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
      # new_text = new_text + " " + stemmer.stem(w)
      
      new_text = new_text + " " + stemmer.stem(w,0,len(w)-1)   # my_stemming

    return new_text

In [ ]:
# Chuyển số thành chữ
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [ ]:
#preprocessing
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [ ]:
processed_text = []
for text in corpus:
  processed_text.append(word_tokenize(str(preprocess(text))))

In [ ]:
print(processed_text[0])

['experiment', 'investig', 'aerodynam', 'wing', 'slipstream', 'experiment', 'studi', 'wing', 'propel', 'slipstream', 'made', 'order', 'determin', 'spanwis', 'distribut', 'lift', 'increas', 'due', 'slipstream', 'differ', 'angl', 'attack', 'wing', 'differ', 'free', 'stream', 'slipstream', 'veloc', 'ratio', 'result', 'intend', 'part', 'evalu', 'basi', 'differ', 'theoret', 'treatment', 'problem', 'compar', 'span', 'load', 'curv', 'togeth', 'support', 'evid', 'show', 'substanti', 'part', 'lift', 'increment', 'produc', 'slipstream', 'due', 'destal', 'boundari', 'layer', 'control', 'effect', 'integr', 'remain', 'lift', 'increment', 'subtract', 'destal', 'lift', 'found', 'agre', 'well', 'potenti', 'flow', 'theori', 'empir', 'evalu', 'destal', 'effect', 'made', 'specif', 'configur', 'experi']


# Vector Space Model

## Retrieve with inverted index

### Building Inverted Index

In [ ]:
from collections import *
import time

term_doc = []

def make_unique(l):
    l = set(l) #remove Duplicates
    l = list(l) #convert to list again
    return l

def index1(l,d):
    occurences = {}
    for word in l:
        d = {}
        for i in range(len(processed_text)):
            temp = [j for j,val in enumerate(processed_text[i]) if val==word]
            if temp :
                d[i+1] = temp
        occurences[word] = d
    return occurences

start = time.time()
for doc in processed_text:
    for j in doc:
        term_doc.append(j)


term_doc = make_unique(term_doc)
posting_list = {}
posting_list = index1(term_doc,posting_list)
end = time.time()

print("Inverted Index has been Built !")
print("Time of Built Inverted Index:\t", (end-start), 'seconds')

Inverted Index has been Built !
Time of Built Inverted Index:	 55.38287830352783 seconds


In [ ]:
print(posting_list)

{'omiss': {262: [101], 1206: [65]}, '12degre': {717: [30]}, 'promin': {403: [53]}, 'flexibl': {42: [18, 27, 43], 78: [13, 28], 100: [102], 401: [188], 637: [32], 749: [25], 781: [21, 76], 783: [73], 826: [132], 895: [34], 919: [61], 1035: [14, 69, 128], 1066: [22, 55], 1087: [66], 1197: [6, 27, 30, 87], 1287: [5, 11, 48], 1289: [139], 1294: [62], 1320: [60], 1321: [1, 20, 40, 57, 63, 66, 154], 1322: [9], 1362: [35]}, 'entranc': {340: [20, 38], 426: [7, 14, 75], 964: [4, 9], 1139: [1, 50]}, 'highli': {215: [2, 12], 245: [75], 262: [80], 272: [25, 70, 147, 201], 283: [15], 293: [17], 297: [12], 315: [170], 329: [258, 334], 334: [64], 364: [37], 447: [3], 565: [109], 576: [112], 625: [151], 638: [6, 31], 712: [39], 757: [61], 900: [17], 1195: [80], 1205: [61, 105], 1215: [3, 35], 1224: [132], 1263: [3, 66], 1281: [26], 1366: [108], 1391: [18, 47]}, 'notic': {406: [74], 605: [79], 773: [113], 799: [135], 1328: [30]}, 'hollow': {818: [12], 1300: [16, 96]}, 'algebra': {34: [64, 102], 111: [3

In [ ]:
import sys
print("Size of posting list:\t",sys.getsizeof(posting_list))

Size of posting list:	 147552


In [ ]:
print("Số lượng term được lựa chọn: ", len(term_doc))

Số lượng term được lựa chọn:  4416


In [ ]:
with open('term.txt', 'w') as file:
  for item in term_doc:
    item = item + '\n'
    file.writelines(item)
file.close()

### Building TF-IDF

In [ ]:
from math import log, pow, sqrt

#dictionaries to hold idf values for words in body 
idf = {}

#dictionaries to hold tf values for words in body for each document
tf = {}

def calc_tf_idf(tf,idf,org,N): 	
    for key,val in org.items():
        raw_tf = {}
        #idf value for token 'key'
        idf[key] = (log((float(N)/len(val.keys())),10))  #val.keys() - in how many titles the wor occurred
        for doc_key,doc_val in val.items():
            if len(doc_val)>0:   #if the word is not in start (title words)
                #tf value for token 'key' and document 'doc_key'
                raw_tf[doc_key] = (1 + log(len(doc_val),10)) * idf[key]
            else:
                raw_tf[doc_key] = 0
        tf[key] = raw_tf

def normalize_doc():
    for i in range(len(processed_text)):
        temp = []
        l = 0.0
        for word in processed_text[i]:
            if word not in temp:
                temp.append(word)
        
        # 1+log(tf); tf -> occurence of word in that document
        # calculating full document to a score
        for word in temp:
          l = l + pow(tf[word][i+1],2)
                
        l = sqrt(l)
        for word in temp:
          tf[word][i] = tf[word][i+1]/l
        return l

In [ ]:
#calculates tf-idf for post
calc_tf_idf(tf, idf, posting_list, len(processed_text))

#normalizes body for all documents
norm = normalize_doc()

print("TF-IDF has been Built !")

TF-IDF has been Built !


In [ ]:
print(tf['wing'])

{1: 1.1699089801901907, 13: 1.2688629079497689, 14: 1.2688629079497689, 30: 1.030441252240303, 31: 1.030441252240303, 42: 1.345617537352208, 52: 1.4613538052119297, 60: 1.1699089801901907, 69: 0.792019596530837, 76: 1.1699089801901907, 78: 1.030441252240303, 92: 0.792019596530837, 95: 1.030441252240303, 97: 0.792019596530837, 146: 1.345617537352208, 147: 1.4613538052119297, 189: 1.2688629079497689, 191: 0.792019596530837, 195: 1.1699089801901907, 199: 1.1699089801901907, 200: 1.345617537352208, 202: 1.030441252240303, 204: 1.345617537352208, 205: 1.5477983638495443, 222: 1.345617537352208, 224: 0.792019596530837, 225: 1.584039193061674, 226: 0.792019596530837, 227: 1.2688629079497689, 229: 1.4613538052119297, 230: 1.5072845636592347, 235: 1.030441252240303, 244: 0.792019596530837, 246: 1.1699089801901907, 247: 1.4613538052119297, 250: 1.2688629079497689, 251: 1.030441252240303, 252: 1.030441252240303, 256: 0.792019596530837, 265: 0.792019596530837, 279: 1.2688629079497689, 284: 1.16990

### Process Query and Retrieve

**Process Query**

In [ ]:
def normalize_query(wt, l):
    # l = 0.0
    # for word in wt.keys():
    #     l = l + pow(wt[word],2)
    # l = sqrt(l)
    for word in wt.keys():
        if l != 0:
            wt[word] = wt[word]/l
        else:
            wt[word] = 0.0

def process_query(_query):
    
    tf_query = {}
    wt_body = {}

    #calculating raw tf
    for token in _query:
        if token not in tf_query:
            tf_query[token] = 1
        else:
            tf_query[token] = tf_query[token] + 1

    #calculating total weight using the logarithmic formula for tf and multiplying with idf 
    for word in tf_query.keys():
        tf_query[word] = 1 + log(tf_query[word],10)

        if word in idf.keys():
            wt_body[word] = tf_query[word]*idf[word]
        else:
            wt_body[word] = 0.0

    #normalizing query vectors to unit vectors for body
    normalize_query(wt_body, norm)

    body_score = [0]*(len(processed_text))
    doc_score = [0]*(len(processed_text))

    #cosine similarity with documents w.r.t body
    for word in wt_body:
        if word in tf.keys():
            for doc in tf[word]:
                try:
                    body_score[doc] += wt_body[word]*tf[word][doc]
                except:
                    pass

    #total document score 
    for i in range(len(doc_score)):
        doc_score[i] = body_score[i]

    return doc_score

In [ ]:
processed_query = []
for text in queries:
  processed_query.append(word_tokenize(str(preprocess(text))))

**Retrieve**

In [ ]:
res = []

start = time.time()
for i in range(len(processed_query)):
    result = process_query(processed_query[i])
    retrieved=[]    

    tmp = []
    for index, val in enumerate(result):
      tmp.append((index, val))
    tmp.sort(key=lambda x:x[1], reverse=True)

    for k in range(len(processed_text)):
      doc_retrieval, score =tmp[k]
      if score > 0:
        retrieved.append(doc_retrieval)

    res.append(retrieved)
end = time.time()

print("Time of Retrieval:\t", (end-start), 'seconds')

Time of Retrieval:	 0.3868544101715088 seconds


In [ ]:
print(res[1])

[12, 14, 184, 746, 51, 78, 202, 100, 781, 486, 172, 1380, 1361, 141, 792, 810, 1089, 1169, 284, 29, 364, 1331, 724, 658, 1170, 875, 1263, 798, 836, 729, 253, 884, 47, 712, 1309, 195, 685, 721, 416, 747, 92, 726, 497, 725, 1147, 390, 1163, 908, 1300, 453, 805, 883, 288, 374, 75, 909, 1095, 1144, 1066, 1332, 1334, 997, 640, 329, 1158, 1012, 846, 804, 395, 1042, 579, 862, 220, 911, 695, 82, 441, 244, 311, 917, 163, 625, 811, 870, 1051, 1246, 606, 76, 700, 873, 876, 209, 1167, 638, 731, 328, 878, 1328, 373, 1217, 1294, 58, 187, 466, 1202, 345, 1379, 1197, 948, 1168, 574, 860, 415, 1320, 896, 101, 293, 33, 1165, 1166, 837, 834, 624, 129, 1212, 85, 1015, 588, 1375, 914, 1002, 70, 189, 701, 702, 1297, 1347, 1303, 251, 925, 581, 607, 1283, 595, 844, 1097, 1195, 1239, 536, 699, 576, 1295, 835, 1035, 160, 554, 1013, 874, 886, 24, 36, 198, 263, 401, 756, 827, 840, 1279, 722, 723, 1157, 812, 25, 1299, 77, 806, 981, 578, 826, 833, 720, 976, 83, 962, 431, 429, 982, 552, 715, 16, 116, 318, 346, 359, 

### Evaluation

In [ ]:
cal_map = Evaluate(evaluate_path, res, rank=True)
iMAP = cal_map.MAP(interpolate=True)
print("Mean Average Precision Interpolation:\t", iMAP)

Mean Average Precision Interpolation:	 0.38573610117130414


In [ ]:
cal_recall_precision = Evaluate(evaluate_path, res, rank=False)
recall = cal_recall_precision.average(cal_recall_precision.recall)
print("Recall:\t", recall)
precision = cal_recall_precision.average(cal_recall_precision.precision)
print("Precision:\t", precision)

Recall:	 0.9610552741851592
Precision:	 0.009700296208288557


## Retrieve with vector

### Preprocessing

In [ ]:
import nltk
nltk.download('gutenberg')
nltk.download('genesis')
nltk.download('inaugural')
nltk.download('nps_chat')
nltk.download('webtext')
nltk.download('treebank')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package genesis to /root/nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package inaugural to /root/nltk_data...
[nltk_data]   Package inaugural is already up-to-date!
[nltk_data] Downloading package nps_chat to /root/nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
[nltk_data] Downloading package webtext to /root/nltk_data...
[nltk_data]   Package webtext is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from nltk.corpus import stopwords

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.book import FreqDist
import pandas as pd
import numpy as np
from math import log10  ## For IDF formula

In [ ]:
cleaned_corpus = []
for doc in corpus:
  data = convert_lower_case(doc)
  data = remove_punctuation(data) #remove comma seperately
  data = remove_apostrophe(data)
  data = remove_stop_words(data)
  data = convert_numbers(data)
  data = stemming(data) #needed again as we need to stem the words
  data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
  data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
  doc_text = ''.join(data)
  cleaned_corpus.append(doc_text)
print(cleaned_corpus)

[' experiment investig aerodynam wing slipstream experiment studi wing propel slipstream made order determin spanwis distribut lift increas due slipstream differ angl attack wing differ free stream slipstream veloc ratio result intend part evalu basi differ theoret treatment problem compar span load curv togeth support evid show substanti part lift increment produc slipstream due destal boundari layer control effect integr remain lift increment subtract destal lift found agre well potenti flow theori empir evalu destal effect made specif configur experi', ' simpl shear flow past flat plate incompress fluid small viscos studi high speed viscou flow past two dimension bodi usual necessari consid curv shock wave emit nose lead edg bodi consequ exist inviscid rotat flow region shock wave boundari layer situat aris instanc studi hyperson viscou flow past flat plate situat somewhat differ prandtl classic boundari layer problem prandtl origin problem inviscid free stream outsid boundari layer

### Convert vector

In [ ]:
vectorizerX = TfidfVectorizer()
vectorizerX.fit(cleaned_corpus)
doc_vector = vectorizerX.transform(cleaned_corpus)
print(vectorizerX.get_feature_names())

print(doc_vector.shape)

['0degre', '100degre', '100x10', '10degre', '10g', '1103a', '11in', '12degre', '1400degreek', '14in', '14x10', '15000degre', '15degre', '15x10', '15x106', '1700f', '18000k', '180degre', '18degre', '18in', '195e', '1degre', '1f', '1x10', '2000degreek', '200degre', '20degre', '20degreec', '24', '2400degre', '25degre', '25x10', '2n', '2p', '2x10', '3000degreek', '30degre', '30th', '3300degreer', '3600degreek', '3800degre', '3ft', '3x10', '4000degreek', '40degre', '45degre', '49degre', '4degre', '4ft', '4th', '4x10', '500degre', '50degre', '55degre', '5d', '5degre', '5th', '6000degreek', '60degre', '63a2xx', '63a4xx', '64a010', '65a004', '65degre', '6in', '6x10', '707a', '75', '75degre', '85degre', '8a', '8n', '8x10', '90degre', '99degre', '9x10', 'a50f06', 'a51j04', 'a52b06', 'ab', 'abbrevi', 'abil', 'abl', 'ablat', 'abrupt', 'abruptli', 'absenc', 'absent', 'absolut', 'absorb', 'absorpt', 'abstract', 'abundantli', 'academ', 'acceler', 'acceleromet', 'accentu', 'accept', 'access', 'acciden

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
df1 = pd.DataFrame(doc_vector.toarray(), columns=vectorizerX.get_feature_names())
df1

,0degre,100degre,100x10,10degre,10g,1103a,11in,12degre,1400degreek,14in,...,zamm,zbrozek,zehnder,zero,zeroth,zhukhovitskii,zone,zoom,zuk,zurich
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.103628,0.0,0.0,0.0,0.0,0.0,0.0
1397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print("Size of document-term matrix:\t", sys.getsizeof(df1))

Size of document-term matrix:	 49459344


### Retrieve

In [ ]:
query_vector = []
for query in queries:
  data = convert_lower_case(query)
  data = remove_punctuation(data) #remove comma seperately
  data = remove_apostrophe(data)
  data = remove_stop_words(data)
  data = convert_numbers(data)
  data = stemming(data) #needed again as we need to stem the words
  data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
  data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
  q = ''.join(data)
  query_vector.append(vectorizerX.transform([q]))

In [ ]:
# calculate cosine similarities
from sklearn.metrics.pairwise import cosine_similarity
cosineSimilarities = []
start = time.time()
for element in query_vector:
  cosineSimilarities.append(cosine_similarity(doc_vector,element).flatten())

res = []
for i in range(len(cosineSimilarities)):
  res_doc = []
  for index, score in enumerate(cosineSimilarities[i]):
    if score > 0:
      res_doc.append((index, score))
  res_doc.sort(key=lambda x:x[1], reverse=True)
  temp = [index+1 for index, _ in res_doc]
  res.append(temp)
end = time.time()

print("Time of Retrieval:\t", (end-start), "seconds")

Time of Retrieval:	 0.6404037475585938 seconds


In [ ]:
print(res[1])

[12, 51, 746, 1169, 884, 184, 100, 253, 141, 14, 810, 429, 92, 792, 862, 47, 726, 1042, 606, 725, 1361, 875, 700, 1263, 909, 497, 834, 1379, 579, 284, 1163, 640, 724, 876, 1089, 1147, 1170, 1158, 1331, 781, 78, 883, 804, 578, 729, 712, 172, 925, 316, 607, 75, 896, 574, 747, 395, 1299, 723, 658, 1380, 836, 1144, 430, 835, 114, 833, 195, 415, 288, 860, 1283, 878, 202, 453, 1309, 911, 672, 251, 1168, 641, 1063, 588, 102, 29, 374, 416, 806, 1320, 486, 69, 722, 285, 805, 730, 1197, 214, 321, 1165, 731, 1012, 264, 77, 378, 1157, 320, 1167, 687, 951, 1217, 1015, 811, 1111, 1341, 2, 886, 33, 364, 390, 650, 1328, 456, 345, 1303, 812, 193, 293, 1002, 220, 52, 654, 837, 1065, 874, 41, 1294, 663, 36, 311, 581, 870, 981, 982, 252, 831, 685, 593, 76, 280, 275, 101, 481, 1295, 82, 561, 552, 1011, 238, 156, 1095, 840, 435, 1300, 997, 798, 373, 328, 576, 58, 1332, 1246, 142, 665, 1377, 1051, 908, 1334, 624, 1155, 516, 410, 226, 720, 594, 263, 969, 446, 1253, 1265, 1166, 846, 1279, 527, 540, 431, 31, 71

### Evaluation

In [ ]:
cal_map = Evaluate(evaluate_path, res, rank=True)
iMAP = cal_map.MAP(interpolate=True)
print("Mean Average Precision Interpolation:\t", iMAP)

Mean Average Precision Interpolation:	 0.4229843389117585


In [ ]:
cal_recall_precision = Evaluate(evaluate_path, res, rank=False)
recall = cal_recall_precision.average(cal_recall_precision.recall)
print("Recall:\t", recall)
precision = cal_recall_precision.average(cal_recall_precision.precision)
print("Precision:\t", precision)

Recall:	 0.9614997186296037
Precision:	 0.009709664197412766


# BM25

## Build Inverted Index and Document Length Table

In [ ]:
class InvertedIndex:

	def __init__(self):
		self.index = dict()

	def __contains__(self, item):
		return item in self.index

	def __getitem__(self, item):
		return self.index[item]

	def add(self, word, docid):
		if word in self.index:
			if docid in self.index[word]:
				self.index[word][docid] += 1
			else:
				self.index[word][docid] = 1
		else:
			d = dict()
			d[docid] = 1
			self.index[word] = d

	#frequency of word in document
	def get_document_frequency(self, word, docid):
		if word in self.index:
			if docid in self.index[word]:
				return self.index[word][docid]
			else:
				raise LookupError('%s not in document %s' % (str(word), str(docid)))
		else:
			raise LookupError('%s not in index' % str(word))

	#frequency of word in index, i.e. number of documents that contain word
	def get_index_frequency(self, word):
		if word in self.index:
			return len(self.index[word])
		else:
			raise LookupError('%s not in index' % word)


class DocumentLengthTable:

	def __init__(self):
		self.table = dict()

	def __len__(self):
		return len(self.table)

	def add(self, docid, length):
		self.table[docid] = length

	def get_length(self, docid):
		if docid in self.table:
			return self.table[docid]
		else:
			raise LookupError('%s not found in table' % str(docid))

	def get_average_length(self):
		sum = 0
		for length in self.table.values():
			sum += length
		return float(sum) / float(len(self.table))


def build_data_structures(corpus):
	idx = InvertedIndex()
	dlt = DocumentLengthTable()
	for docid in corpus:

		#build inverted index
		for word in corpus[docid]:
			idx.add(str(word), str(docid))

		#build document length table
		length = len(corpus[str(docid)])
		dlt.add(docid, length)
	return idx, dlt


In [ ]:
invertedIndex = InvertedIndex()
documentLengthTable = DocumentLengthTable()

start = time.time()
for i, value in enumerate(processed_text):
  for w in processed_text[i]:
    invertedIndex.add(str(w), str(i))

  length = len(processed_text[i])
  documentLengthTable.add(i, length)
end = time.time()
print("Time of Built Inverted Index;\t", (end-start), 'seconds')
print("Size of Inverted Index:\t", sys.getsizeof(invertedIndex))

Time of Built Inverted Index;	 0.13528037071228027 seconds
Size of Inverted Index:	 48


## Ranking

In [ ]:
from math import log

k1 = 1.5
k2 = 100
b = 0.75
R = 0.0


def score_BM25(n, f, qf, r, N, dl, avdl):
	K = compute_K(dl, avdl)
	first = log( ( (float(r) + 0.5) / (float(R) - float(r) + 0.5) ) / ( (float(n) - float(r) + 0.5) / (N - n - R + r + 0.5)), 10)
	second = ((k1 + 1) * f) / (K + f)
	third = ((k2+1) * qf) / (k2 + qf)
	return first * second * third


def compute_K(dl, avdl):
	return k1 * ((1-b) + b * (float(dl)/float(avdl)) )

## Query Processor

In [ ]:

# from invdx import build_data_structures
# from rank import score_BM25
import operator


class QueryProcessor:
	def __init__(self, queries, corpus):
		self.queries = queries
		self.index, self.dlt = build_data_structures(corpus)

	def run(self):
		results = []
		for query in self.queries:
			results.append(self.run_query(query))
		return results

	def run_query(self, query):
		query_result = dict()
		for term in query:
			if term in self.index:
				doc_dict = self.index[term] # retrieve index entry
				for docid, freq in doc_dict.items(): #for each document and its word frequency
					score = score_BM25(n=len(doc_dict), f=freq, qf=1, r=0.0, N=len(self.dlt),
									   dl=self.dlt.get_length(docid), avdl=self.dlt.get_average_length()) # calculate score
					if docid in query_result: #this document has already been scored once
						query_result[docid] += score
					else:
						query_result[docid] = score
		return query_result


In [ ]:
# Process Query
query = []
for text in queries:
  query.append(word_tokenize(str(preprocess(text))))

In [ ]:
corpus_dict = {}
for id, value in enumerate(processed_text):
  corpus_dict[str(id)] = value

In [ ]:
print(corpus_dict['0'])

['experiment', 'investig', 'aerodynam', 'wing', 'slipstream', 'experiment', 'studi', 'wing', 'propel', 'slipstream', 'made', 'order', 'determin', 'spanwis', 'distribut', 'lift', 'increas', 'due', 'slipstream', 'differ', 'angl', 'attack', 'wing', 'differ', 'free', 'stream', 'slipstream', 'veloc', 'ratio', 'result', 'intend', 'part', 'evalu', 'basi', 'differ', 'theoret', 'treatment', 'problem', 'compar', 'span', 'load', 'curv', 'togeth', 'support', 'evid', 'show', 'substanti', 'part', 'lift', 'increment', 'produc', 'slipstream', 'due', 'destal', 'boundari', 'layer', 'control', 'effect', 'integr', 'remain', 'lift', 'increment', 'subtract', 'destal', 'lift', 'found', 'agre', 'well', 'potenti', 'flow', 'theori', 'empir', 'evalu', 'destal', 'effect', 'made', 'specif', 'configur', 'experi']


In [ ]:
start =time.time()
proc = QueryProcessor(query, corpus_dict)
results = proc.run()
end = time.time()
print("Time of Retrieval:\t", (end-start), "seconds")

Time of Retrieval:	 27.392160654067993 seconds


In [ ]:
results[0]
sorted_res = []
for result in results:
  sorted_x = sorted(result.items(), key=operator.itemgetter(1))
  sorted_x.reverse()
  sorted_res.append(sorted_x[:len(sorted_x) // 2])

print(len(sorted_res[0]))
print((sorted_res[0]))


417
[('50', 9.410767271130513), ('485', 8.629376922400201), ('11', 7.8864162940511235), ('183', 7.566524335125464), ('877', 7.056334539747635), ('572', 7.0384566586687125), ('664', 5.999501002674914), ('943', 5.452578573390671), ('745', 5.324513640090738), ('140', 5.204107263086249), ('77', 4.949003495295303), ('878', 4.890158088119206), ('12', 4.883852338662383), ('1360', 4.759535902790979), ('13', 4.665812854890066), ('746', 4.648397916866761), ('452', 4.588127460043601), ('358', 4.569296623018751), ('218', 4.515276072644735), ('171', 4.468679914796323), ('434', 4.344841966930498), ('662', 4.219319904162637), ('875', 4.208011286558204), ('328', 4.178486405172619), ('55', 4.129588667125864), ('1002', 4.1292290438762524), ('1143', 4.056171609416075), ('251', 4.0196888112770655), ('874', 4.015625256681846), ('331', 3.9836362901387106), ('373', 3.9578545778775944), ('1267', 3.948434138932252), ('1339', 3.9246560453606847), ('1185', 3.8044509643493227), ('1262', 3.800916594941293), ('201'

In [ ]:
#results[0].keys()
res = []
for i in range(len(sorted_res)):
  a = []
  for j in range(len(sorted_res[i])):
    a.append(int(sorted_res[i][j][0]))
  res.append(a)

#print(len(res[0]))
print(res[0])

[50, 485, 11, 183, 877, 572, 664, 943, 745, 140, 77, 878, 12, 1360, 13, 746, 452, 358, 218, 171, 434, 662, 875, 328, 55, 1002, 1143, 251, 874, 331, 373, 1267, 1339, 1185, 1262, 201, 791, 194, 1327, 728, 684, 28, 413, 575, 525, 292, 1334, 103, 490, 252, 1168, 1193, 304, 1033, 583, 718, 639, 279, 628, 780, 1146, 1162, 810, 1337, 1041, 1154, 377, 24, 981, 219, 725, 873, 1157, 1304, 605, 327, 587, 816, 282, 706, 100, 314, 239, 798, 924, 494, 299, 1299, 344, 496, 591, 414, 907, 101, 215, 1245, 1218, 716, 1211, 1379, 1100, 250, 600, 584, 657, 1183, 1197, 203, 301, 310, 817, 870, 1180, 141, 74, 1166, 521, 757, 883, 192, 480, 1112, 308, 208, 1, 916, 710, 363, 1167, 637, 99, 428, 1314, 1088, 455, 159, 1124, 1127, 91, 492, 680, 1148, 634, 81, 1190, 1109, 990, 541, 293, 545, 429, 1330, 283, 102, 799, 1036, 1169, 394, 1063, 315, 539, 234, 83, 859, 624, 415, 35, 305, 213, 571, 430, 164, 353, 913, 1011, 1050, 291, 1062, 49, 1047, 754, 61, 685, 1164, 1281, 1361, 1120, 551, 1200, 1254, 409, 629, 1065,

In [ ]:
for i in range(len(res)):
  for j in range(len(res[i])):
    res[i][j] += 1

print(res[0]) 

[51, 486, 12, 184, 878, 573, 665, 944, 746, 141, 78, 879, 13, 1361, 14, 747, 453, 359, 219, 172, 435, 663, 876, 329, 56, 1003, 1144, 252, 875, 332, 374, 1268, 1340, 1186, 1263, 202, 792, 195, 1328, 729, 685, 29, 414, 576, 526, 293, 1335, 104, 491, 253, 1169, 1194, 305, 1034, 584, 719, 640, 280, 629, 781, 1147, 1163, 811, 1338, 1042, 1155, 378, 25, 982, 220, 726, 874, 1158, 1305, 606, 328, 588, 817, 283, 707, 101, 315, 240, 799, 925, 495, 300, 1300, 345, 497, 592, 415, 908, 102, 216, 1246, 1219, 717, 1212, 1380, 1101, 251, 601, 585, 658, 1184, 1198, 204, 302, 311, 818, 871, 1181, 142, 75, 1167, 522, 758, 884, 193, 481, 1113, 309, 209, 2, 917, 711, 364, 1168, 638, 100, 429, 1315, 1089, 456, 160, 1125, 1128, 92, 493, 681, 1149, 635, 82, 1191, 1110, 991, 542, 294, 546, 430, 1331, 284, 103, 800, 1037, 1170, 395, 1064, 316, 540, 235, 84, 860, 625, 416, 36, 306, 214, 572, 431, 165, 354, 914, 1012, 1051, 292, 1063, 50, 1048, 755, 62, 686, 1165, 1282, 1362, 1121, 552, 1201, 1255, 410, 630, 1066

## Evaluation

In [ ]:
cal_map = Evaluate(evaluate_path, res, rank=True)
iMAP = cal_map.MAP(interpolate=True)
print("Mean Average Precision Interpolation:\t", iMAP)

Mean Average Precision Interpolation:	 0.44659667072789716


In [ ]:
cal_recall_precision = Evaluate(evaluate_path, res, rank=False)
recall = cal_recall_precision.average(cal_recall_precision.recall)
print("Recall:\t", recall)
precision = cal_recall_precision.average(cal_recall_precision.precision)
print("Precision:\t", precision)

Recall:	 0.9067322153722316
Precision:	 0.018071594830578817


# Evaluation Lucene

## Lucene + TF-IDF

In [61]:
path_lucene = '/content/drive/MyDrive/CS419.N11.KHCL/Final_Project/Lucene/output/TF-IDF.txt'
data = []
with open(path_lucene, 'r') as f:
  dataset = f.readlines()
  for item in dataset:
    data.append(item.split())

res = []
temp = []
t = 0
for i in range(len(data)):
  if t+1 != int(data[i][0]):
    res.append(temp)
    t += 1
    temp = []
    temp.append(int(data[i][2]))
  else:
    temp.append(int(data[i][2]))

res.append(temp)

In [62]:
print(res[1])

[12, 746, 792, 810, 700, 141, 51, 92, 1379, 1380, 47, 606, 725, 909, 429, 100, 1169, 884, 712, 316, 395, 253, 1089, 497, 875, 1246, 781, 747, 883, 202, 911, 1063, 70, 1331, 1011, 1361, 1163, 78, 1012, 876, 1008, 724, 184, 1197, 430, 142, 435, 1095, 731, 52, 1147, 862, 195, 1111, 321, 835, 1007, 75, 1250, 1303, 203, 834, 264, 1327, 1005, 101, 804, 76, 288, 882, 1003, 798, 715, 1006, 1170, 204, 756, 654, 658, 650, 640, 275, 209, 726, 391, 1041, 925, 1009, 574, 880, 833, 302, 979, 441, 1392, 1158, 762, 1004, 251, 14, 648, 837, 945, 1217, 345, 805, 1377, 322, 723, 284, 320, 552, 193, 641, 416, 315, 836, 981, 1144, 799, 1165, 83, 874, 593, 222, 1328, 187, 373, 1300, 1224, 1002, 1124, 172, 406, 854, 1103, 285, 685, 663, 238, 1010, 93, 364, 198, 860, 1314, 1042, 422, 803, 266, 59, 997, 878, 578, 914, 350, 431, 865, 614, 1240, 478, 425, 1000, 414, 415, 225, 1167, 665, 1155, 675, 924, 934, 1209, 540, 1130, 720, 179, 927, 449, 1320, 368, 33, 309, 239, 607, 615, 280, 707, 476, 1343, 420, 1084, 10

In [63]:
cal_map = Evaluate(evaluate_path, res, rank=True)
iMAP = cal_map.MAP(interpolate=True)
print("Mean Average Precision Interpolation:\t", iMAP)

Mean Average Precision Interpolation:	 0.4046704845666254


In [64]:
cal_recall_precision = Evaluate(evaluate_path, res, rank=False)
recall = cal_recall_precision.average(cal_recall_precision.recall)
print("Recall:\t", recall)
precision = cal_recall_precision.average(cal_recall_precision.precision)
print("Precision:\t", precision)

Recall:	 0.9675114965727328
Precision:	 0.007879531405975264


## Lucene + BM25

In [57]:
path_lucene = '/content/drive/MyDrive/CS419.N11.KHCL/Final_Project/Lucene/output/BM25.txt'
data = []
with open(path_lucene, 'r') as f:
  dataset = f.readlines()
  for item in dataset:
    data.append(item.split())

res = []
temp = []
t = 0
for i in range(len(data)):
  if t+1 != int(data[i][0]):
    res.append(temp)
    t += 1
    temp = []
    temp.append(int(data[i][2]))
  else:
    temp.append(int(data[i][2]))

res.append(temp)

In [58]:
print(res[1])

[12, 746, 792, 51, 141, 810, 700, 1380, 1089, 92, 1169, 100, 184, 78, 909, 14, 606, 724, 725, 712, 47, 1379, 781, 1361, 747, 875, 1170, 253, 172, 497, 883, 884, 395, 202, 1331, 195, 251, 1012, 876, 75, 1147, 911, 658, 726, 486, 429, 1246, 1263, 1163, 284, 836, 1095, 1158, 862, 36, 1197, 288, 70, 640, 416, 834, 574, 374, 101, 316, 805, 1063, 1042, 1217, 804, 729, 29, 209, 1309, 1303, 321, 1300, 833, 364, 731, 870, 453, 52, 76, 835, 878, 142, 837, 391, 997, 441, 588, 798, 1144, 1328, 925, 685, 860, 1011, 345, 293, 435, 1002, 1111, 82, 908, 264, 896, 430, 756, 1320, 203, 1008, 981, 1051, 715, 552, 1167, 650, 1268, 1003, 914, 204, 220, 58, 311, 193, 578, 415, 638, 886, 979, 373, 315, 33, 431, 607, 1072, 846, 799, 672, 28, 828, 593, 1007, 663, 1165, 187, 762, 1294, 945, 406, 695, 275, 280, 654, 882, 812, 302, 1068, 1377, 874, 1295, 244, 1250, 624, 579, 1168, 285, 723, 436, 1392, 1124, 198, 1299, 42, 466, 1005, 1155, 329, 328, 1327, 262, 581, 720, 378, 1015, 554, 982, 702, 309, 893, 390, 721

In [59]:
cal_map = Evaluate(evaluate_path, res, rank=True)
iMAP = cal_map.MAP(interpolate=True)
print("Mean Average Precision Interpolation:\t", iMAP)

Mean Average Precision Interpolation:	 0.44852678403569135


In [60]:
cal_recall_precision = Evaluate(evaluate_path, res, rank=False)
recall = cal_recall_precision.average(cal_recall_precision.recall)
print("Recall:\t", recall)
precision = cal_recall_precision.average(cal_recall_precision.precision)
print("Precision:\t", precision)

Recall:	 0.9749047420738428
Precision:	 0.00793730918375304
